<a href="https://colab.research.google.com/github/hakuban-create/Chest_xray_Project_4/blob/main/xray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Dependencies to Visualize the model
%matplotlib inline
from IPython.display import Image, SVG
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(0)


In [2]:
# Filepaths, numpy, and Tensorflow
import os
import cv2
from google.colab import drive
import numpy as np
import tensorflow as tf

In [3]:
# Sklearn scaling
from sklearn.preprocessing import MinMaxScaler

In [4]:
# Keras
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense
from io import BytesIO
from tensorflow.keras.preprocessing import image

In [5]:
drive.mount('/content/drive')

data_directory_path = os.path.join("/content/drive/MyDrive/chest_xray/chest_xray")
train_directory_path = os.path.join(data_directory_path, "train")
train_normal_path = os.path.join(train_directory_path, "NORMAL")
train_pneumonia_path = os.path.join(train_directory_path, "PNEUMONIA")

test_directory_path = os.path.join(data_directory_path, "test")
test_normal_path = os.path.join(test_directory_path, "NORMAL")
test_pneumonia_path = os.path.join(test_directory_path, "PNEUMONIA")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


----- LOADING DATASETS -----

In [6]:
print(len(os.listdir(train_normal_path)))
print(len(os.listdir(train_pneumonia_path)))
print(len(os.listdir(test_normal_path)))
print(len(os.listdir(test_pneumonia_path)))


1341
3925
234
390


In [7]:
X_train = []
y_train = []

X_test = []
y_test = []

In [8]:
for image_file in os.listdir(train_normal_path):
  image_path = os.path.join(train_normal_path, image_file)
  image = cv2.imread(image_path)
  image = cv2.resize(image, (256, 256))
  X_train.append(image)
  y_train.append(0)

In [ ]:
for image_file in os.listdir(train_pneumonia_path):
  image_path = os.path.join(train_pneumonia_path, image_file)
  image = cv2.imread(image_path)
  image = cv2.resize(image, (256, 256))
  X_train.append(image)
  y_train.append(1)

In [ ]:
for image_file in os.listdir(test_normal_path):
  image_path = os.path.join(test_normal_path, image_file)
  image = cv2.imread(image_path)
  image = cv2.resize(image, (256, 256))
  X_test.append(image)
  y_test.append(0)

In [ ]:
for image_file in os.listdir(test_pneumonia_path):
  image_path = os.path.join(test_pneumonia_path, image_file)
  image = cv2.imread(image_path)
  image = cv2.resize(image, (256, 256))
  X_test.append(image)
  y_test.append(1)

In [ ]:
# Convert lists to NumPy arrays
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

In [ ]:
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

In [ ]:
# Plot the first image from the dataset
plt.imshow(X_train[0], cmap=plt.cm.Greys)

----- BUILDING THE MODEL -----


In [ ]:
model = Sequential()
model.add(Dense(100, activation='relu', input_shape=(256, 256, 1)))
model.add(Dense(100, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(
    X_train,
    y_train,
    epochs=10,
    shuffle=True,
    verbose=2
)

In [ ]:
model_loss, model_accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

------- TEST THE MODEL ---------

In [ ]:
test = np.expand_dims(X_train[0], axis=0)
plt.imshow(scaler.inverse_transform(test).reshape(28, 28), cmap=plt.cm.Greys)
model.predict(test).round()